In [2]:
from random import randint
from time import sleep
import yfinance as yf
import openpyxl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
def ticker(csv_file, ticker_col):
    df = pd.read_csv(csv_file)
    df_tick = df[ticker_col].drop_duplicates(keep='first').dropna()
    return df_tick

In [8]:
ticker("financial_ticker_largest.csv", "ticker")

0          A
1         AA
2       AABA
3        AAC
4       AADR
        ... 
6363     ZTS
6364    ZUMZ
6365     ZUO
6366    ZYME
6367    ZYNE
Name: ticker, Length: 6368, dtype: object

In [12]:
df_tick = ticker('financials ticker SnP500.csv', 'Symbol')

In [16]:
df_tick2 = np.array_split(df_tick, 2)
df_tick2

[0       MMM
 1       AOS
 2       ABT
 3      ABBV
 4       ACN
        ... 
 248     ICE
 249     IBM
 250      IP
 251     IPG
 252     IFF
 Name: Symbol, Length: 253, dtype: object,
 253    INTU
 254    ISRG
 255     IVZ
 256     IQV
 257     IRM
        ... 
 500     XYL
 501     YUM
 502     ZBH
 503    ZION
 504     ZTS
 Name: Symbol, Length: 252, dtype: object]

In [ ]:
def get_input(): 
    comp_input = input("What company do you want to check? ")
    comp_input = comp_input.lower()
    return comp_input
song_input = get_input()

#Check if song is in large DB/ if input makes any sense
def comp_check():
    comp_check = df_spotify[df_spotify['track_name'].str.contains(song_input)]
    if len(song_check0) > 0:
        return song_check0 
    else:
        get_input()
song_check0 = song_check_overall()

In [ ]:
# Selection idea
def user_selection():
    for index, item in enumerate(options):
                print(f"Option {index + 1} - {item}")
            while True:
                try: # Is used to avoid errors
                    choice = int(input("Selection: "))
                    if choice >= 1 and choice <= len(options):
                        return options[choice-1]
                except:
                    continue

In [ ]:
# df['symbol']
# user input = 'AAPL'
# df.loc[df['symbol']==user input]
# if len(df.loc[df['symbol']==user input]) ==1
#    print('The following company was found:')
#    elif len(df.loc[df['symbol']==user input]) > 1
# df = df.loc